In [3]:
import torch
import pandas as pd
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
df = pd.read_csv("/content/drive/MyDrive/EmbedTurk/dataset/new_dataset2.csv")

In [13]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct" do not use

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
)

def summarize_reviews(reviews, model, tokenizer):

  messages = [
        {"role": "system", "content": """Bir yapay zeka asistanı olarak sana bir
        ürün hakkında kullanıcı yorumları vereceğim.
        Görevin, bu yorumları analiz ederek temel noktaları ve genel duyguları özetlemektir.
        Yorumları inceleyerek genel geri bildirimin kısa bir özetini yap, öne çıkan
        olumlu yönleri ve güçlü yanları vurgula, belirgin olumsuz yönleri ve zayıf
        noktaları belirt ve yorumlar arasında tekrar eden ortak temaları veya kalıpları
        tespit et. Tüm bu istenilenleri sadece 1-2 cümleyle Türkçe olarak anlat.
        Ürün dışında kalan gereksiz detaylara (hediye alma amacı, alıcı profili vb.) yer verme."""},
        {"role": "user", "content": reviews}
    ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      pad_token_id=tokenizer.eos_token_id,
      max_new_tokens=512,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]

  return tokenizer.decode(response, skip_special_tokens=True)


def get_product_reviews(df, model, tokenizer):
    summaries = []

    for index, row in df.iterrows():
        product_name = row["product_name"]
        product_comments = row["comments"]

        if len(product_comments) > 2:
          cleaned_text = re.sub(r'[^\w\s,.!?:;@#&%*()\-\'"€£₺]+', '', product_comments)
          cleaned_comments = re.sub(r"['\"]", "", cleaned_text)

          concatenated_reviews = " ".join([review.strip() for review in cleaned_comments.split(",")])

          product_summary = summarize_reviews(concatenated_reviews, model, tokenizer)

          summaries.append(product_summary)

        else:
          summaries.append('yok')

    df["summary"] = summaries

    return df


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
df_test = get_product_reviews(df.head(10), model, tokenizer)

# df_test.to_csv("/content/drive/MyDrive/EmbedTurk/dataset/summarized_reviews.csv", index=False)

df_test.head()

<ipython-input-13-ab8c5b2f7e59>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["summary"] = summaries


,brand,product_name,category,price,rating,info,comments,summary
0,MAF Atölye,Altın Kaplama Paslanmaz Çelik Kolye,Kadın Aksesuar,285 TL,-,- Doğal inci taşı kullanımı sayesinde hem este...,[],yok
1,byebruketenci,Turuncu Yün Bere Tilki Punch Yılbaşı Hediye De...,Kadın Aksesuar,340 TL,4.9,Yünlü yapısı sayesinde soğuk havalarda başınız...,['Ağlıcam çok güzel 🥹 Keşke biraz daha ucuz ol...,"""Ürün, genel olarak olumlu yorumlar alıyor. Ku..."
2,Calvin Klein,Erkek Cüzdan 31ck220001-black,Erkek Aksesuar,1.899 TL,3.0,Gerçek deri materyaliyle hem şık hem de dayanı...,[],yok
3,Casio,Mtp-v002g-7b2udf Erkek Kol Saati,Erkek Aksesuar,"1.015,99 TL",4.8,Takvim özelliği ile tarih bilgisine kolayca er...,"['Babam için aldim beğendi.', 'kaliteli ürün',...","""Ürün, yüksek kaliteli ve şık bir saat olarak ..."
4,Kiğılı,Cüzdan,Erkek Aksesuar,"525,99 TL",4.6,Trendleri takip eden tasarımları ile fark yara...,"['Eşime hediye almıştım çok beğendi', 'hediye ...",Ürün genel olarak çok beğeniliyor ve kaliteli ...


In [15]:
df_test["summary"][1]

'"Ürün, genel olarak olumlu yorumlar alıyor. Kullanıcılar, ürünün kaliteli, tatlı ve sıcak tutan yönlerini beğeniyor. Renk seçenekleri de çok iyi karşılanıyor. Bazı kullanıcılar, fiyatın biraz yüksek olduğunu belirtiyor, ancak genel olarak ürünün kalitesine değer veriyor. Yorumlar arasında tekrar eden bir tema, ürünün tasarımı ve kalitesi olduğudur."'

In [33]:
#info kısmını temizle
df_test["info"][8]

'Model Kodu: GUGW0629G1 Cinsiyet: Erkek Kordon Rengi: Metalik Gri Kordon Tipi: Hasır Tarz: Klasik Saatler Teknoloji: Quartz Kasa Çapı: 44 mm Kasa Kalınlığı: 9 mm Kasa Rengi: Metalik Gri Kasa Şekli: Yuvarlak Cam Özellik: Mineral Su Geçirmezlik: 3 ATM Kadran Tipi: Analog Kadran Rengi: Siyah Revision: 1695988524'